# 优化模型

## 集成算法

集成算法是提高准确度的有效方法之一，主要有以下几种：
- 袋装Bagging算法
    - 先将训练集分离成多个子集，然后通过各个子集训练多个模型
- 提升Boosting算法
    - 训练多个模型并组成一个序列，序列中的每一个模型都会修正前一个模型的错误
- 投票Voting算法
    - 训练多个模型，并采用样本统计来提高模型的准确度


In [3]:
# 袋装算法：提高分类准确率，通过给定组合的方式获得最优解
# 三种模型：袋装决策树（Bagged Decision Trees）；随机森林（Random Forest）；极端随机树（Extra Trees）

# 袋装随机树：在方差很大时非常有效

import pandas as pd
from sklearn.model_selection import KFold,cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore', 'RuntimeWarning')

filename = 'pima_data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_csv(filename, names=names)
# 将数据分为输入数据和输出结果
array = data.values
X = array[:, 0:8]
Y = array[:, 8]
num_folds = 10
seed = 7
kfold = KFold(n_splits=num_folds, random_state=seed)
cart = DecisionTreeClassifier()
num_tree = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_tree, random_state=seed)
result = cross_val_score(model, X, Y, cv=kfold)
result.mean()

0.770745044429255

In [4]:
# 随机森林

from sklearn.ensemble import RandomForestClassifier

max_features = 3
model = RandomForestClassifier(n_estimators=num_tree, random_state=seed, max_features=max_features)
result = cross_val_score(model, X, Y, cv=kfold)
result.mean()

0.7733766233766234

In [5]:
# 极端随机树
# 与随机森林的两个主要区别
# 1、随机森林用的是Bagging模型；极端随机树是每棵决策树应用的是相同的全部训练样本
# 2、随机森林是在一个随机子集内得到最优分叉特征属性；极端树是完全随机地选择分叉属性特征

from sklearn.ensemble import ExtraTreesClassifier
max_features = 7
model = ExtraTreesClassifier(n_estimators=num_tree, random_state=seed, max_features=max_features)
result = cross_val_score(model, X, Y, cv=kfold)
result.mean()

0.762987012987013

In [6]:
# 提升算法：提高弱分类算法准确度的方法
# 常见两种：1、AdaBoost；2、随机梯度提升 Stochastic Gradient Boosting

# AdaBoost
from sklearn.ensemble import AdaBoostClassifier

num_tree = 30
model = AdaBoostClassifier(n_estimators=num_tree, random_state=seed)
result = cross_val_score(model, X, Y, cv=kfold)
result.mean()

0.760457963089542

In [7]:
# 随机梯度提升 GBM：要找到某个函数最大值，最好的办法是沿着该函数的梯度方向探寻
from sklearn.ensemble import GradientBoostingClassifier

num_tree = 100
model = GradientBoostingClassifier(n_estimators=num_tree, random_state=seed)
result = cross_val_score(model, X, Y, cv=kfold)
result.mean()

0.7681989063568012

In [12]:
# 投票算法 Voting：多个机器学习算法的集成算法，通过增加预测结果权重 ，提高算法准确度
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings('ignore')

models = []
model_logistic = LogisticRegression()
models.append(('logistic', model_logistic))
model_cart = DecisionTreeClassifier()
models.append(('cart', model_cart))
model_svc = SVC()
models.append(('svm', model_svc))
ensemble_model = VotingClassifier(estimators=models)
result = cross_val_score(ensemble_model, X, Y, cv=kfold)
result.mean()

0.7329801777170197